## Лабораторная работа № 1 
## Выполнение разведочного анализа больших данных с использованием фреймворка Apache Spark

### Часть 1

В данной части работы рассмотрены:
* загрузка данных из HDFS;
* базовые преобразования данных;
* загрузка преобразованных данных в таблицу `Apache Airflow`.

Подключим необходимые библиотеки.

In [1]:
import os
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf
from pyspark.sql.functions import (
    regexp_replace,
    regexp_extract_all,
    col,
    lit
)

Сформируем объект конфигурации для `Apache Spark`, указав необходимые параметры.

In [2]:
def create_spark_configuration() -> SparkConf:
    """
    Создает и конфигурирует экземпляр SparkConf для приложения Spark.

    Returns:
        SparkConf: Настроенный экземпляр SparkConf.
    """
    # Получаем имя пользователя
    user_name = os.getenv("USER")
    
    conf = SparkConf()
    conf.setAppName("lab 1 Test")
    conf.setMaster("yarn")
    conf.set("spark.submit.deployMode", "client")
    conf.set("spark.executor.memory", "12g")
    conf.set("spark.executor.cores", "8")
    conf.set("spark.executor.instances", "2")
    conf.set("spark.driver.memory", "4g")
    conf.set("spark.driver.cores", "2")
    conf.set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0")
    conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog")
    conf.set("spark.sql.catalog.spark_catalog.type", "hadoop")
    conf.set("spark.sql.catalog.spark_catalog.warehouse", f"hdfs:///user/{user_name}/warehouse")
    conf.set("spark.sql.catalog.spark_catalog.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")

    return conf

Создаём сам объект конфигурации.

In [3]:
conf = create_spark_configuration()

Создаём и выводим на экран сессию `Apache Spark`. В процессе создания сессии происходит подключение к кластеру `Apache Hadoop`, что может занять некоторое время.

In [ ]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

Для исследования будем использовать датасет `"US Used cars dataset"`, расположенный на платформе `Kaggle` по адресу https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset.

Датасет включает в себя информацию о более чем трех миллионах используемых машин в США. Он разрешен для использования в учебных целях.

Данный датасет уже загружен в `HDFS` по адресу: `hdfs:///datasets/used_cars_data.csv`

Указываем путь в `HDFS` для файла с данными.

In [5]:
path = "hdfs:///datasets/used_cars_data.csv"

Заполняем датафрейм данными из файла.

In [6]:
df = (spark.read.format("csv")
      .option("header", "true")
      .load(path)
)

Выводим фрагмент датафрейма на экран.

In [ ]:
df.show()

Очевидно, что в целях сохранения ясности изложения и сокращения расчетного времени имеет смысл рассматривать не все солбцы датасета. Оставим следующие колонки, удалив остальные:

| Название столбца  | Расшифровка |
| ------------- | ------------- |
| vin               | Идентификационный номер автомобиля  |
| body_type         | Тип кузова автомобиля (кабриолет, хэтчбек, седан и т.д.)  |
| daysonmarket      | Количество дней, прошедших с момента первого размещения автомобиля на сайте |
| fleet             | Был ли автомобиль ранее частью автопарка |
| has_accidents     | Имеется ли регистрация аварий на данном автомобиле |
| horsepower        | Мощность двигателя в лошадиных силах |
| is_certified      | Является ли автомобиль сертифицированным (на сертифицированные автомобили распространяется гарантийный срок) |
| is_cpo            | Подержанные автомобили, сертифицированные дилером |
| is_oemcpo         | Подержанные автомобили, сертифицированные производителем |
| major_options     | Список основный опций автомобиля |
| maximum_seating   | Максимальное количество посадочных мест |
| mileage           | Величина пробега автомобиля |
| price             | Цена автомобиля |
| wheel_system      | Тип привода |
| year              | Год выпуска автомобиля |

In [11]:
df = df.select(
    "vin", "body_type", "daysonmarket", "fleet", "has_accidents", "horsepower",
    "is_certified", "is_cpo", "is_oemcpo", "major_options",
    "maximum_seating", "mileage", "price", "wheel_system", "year"
)

In [ ]:
df.show()

Выведем на экран метаданные датасета.

In [ ]:
df.printSchema()

Видно, что все столбцы датасета содержат строковый тип данных, что не соответствует ожиданиям. Выполним преобразования типов данных некоторых столбцов.

In [14]:
def transform_dataframe(data: DataFrame) -> DataFrame:
    """
    Преобразует столбцы DataFrame в указанные типы данных и
    выполняет необходимые преобразования.

    Args:
        data (DataFrame): Исходный DataFrame.

    Returns:
        DataFrame: Преобразованный DataFrame.
    """
    # Преобразуем столбцы в соответствующие типы данных
    data = data.withColumn("daysonmarket",
                           col("daysonmarket").cast("Integer"))
    data = data.withColumn("fleet",
                           col("fleet").cast("Boolean"))
    data = data.withColumn("has_accidents",
                           col("has_accidents").cast("Boolean"))
    data = data.withColumn("horsepower",
                           col("horsepower").cast("Float"))
    data = data.withColumn("is_certified",
                           col("is_certified").cast("Boolean"))
    data = data.withColumn("is_cpo",
                           col("is_cpo").cast("Boolean"))
    data = data.withColumn("is_oemcpo",
                           col("is_oemcpo").cast("Boolean"))

    # Преобразуем строку в массив строк
    data = data.withColumn("major_options",
                           regexp_extract_all(col("major_options"),
                                              lit(r"'([^']*)'"),
                                              1)
    )

    # Убираем единицы измерения
    data = data.withColumn("maximum_seating",
                           regexp_replace(col("maximum_seating"),
                                          r"\s+seats",
                                          "").cast("Integer")
    )

    data = data.withColumn("mileage",
                           col("mileage").cast("Float"))
    data = data.withColumn("price",
                           col("price").cast("Float"))
    data = data.withColumn("year",
                           col("year").cast("Integer"))

    return data

In [15]:
df = transform_dataframe(df)

In [ ]:
df.show()

In [ ]:
df.printSchema()

Видно, что теперь столбцы датафрейма содержат значения корректных типов.

Полученный датафрейм сохраним для дальнейшего использования. Сохранение выполним в таблицу `Apache Iceberg`. 

`Apache Iceberg` — это поддерживающий высокую производительность табличный формат для больших данных.

Сначала создадим базу данных, в которой будет расположена таблица. Во избежание путаницы, **каждую таблицу следует называть с использованием фамилии студента**.

In [21]:
database_name = "ivanov_database"

Создадим инструкцию SQL для добавления базы данных в каталог `Apache Spark`.

In [22]:
create_database_sql = f"""
CREATE DATABASE IF NOT EXISTS spark_catalog.{database_name}
"""

In [ ]:
spark.sql(create_database_sql)

Установим созданную базу данных как текущую.

In [24]:
spark.catalog.setCurrentDatabase(database_name)

И, наконец, записываем преобразованный датафрейм в таблицу `sobd_lab1_table`.

In [25]:
# Сохранение DataFrame в виде таблицы
df.writeTo("sobd_lab1_table").using("iceberg").create()

После успешной записи можно посмотреть, какие таблицы входят в базу данных.

In [ ]:
for table in spark.catalog.listTables():
    print(table.name)

Обратите внимание, что при необходимости созданные базу данных и таблицу можно удалить следующими командами.

In [20]:
# spark.sql("DROP TABLE spark_catalog.ivanov_database.sobd_lab1_table")
# spark.sql("DROP DATABASE spark_catalog.ivanov_database")

После успешной записи таблицы останавливаем сессию `Apache Spark`.

In [30]:
spark.stop()